In [ ]:
!pip install scikit-learn PyMuPDF spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import fitz  # PyMuPDF
import os
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

nlp = spacy.load("en_core_web_sm")


In [ ]:
from google.colab import files

print("Upload a ZIP file of multiple resumes in PDF format")
uploaded = files.upload()  # Upload zip file
resume_zip = list(uploaded.keys())[0]

# Extract resumes
with zipfile.ZipFile(resume_zip, 'r') as zip_ref:
    zip_ref.extractall("resumes")


👉 Upload a ZIP file of multiple resumes in PDF format


Saving Zipped PDF Files.zip to Zipped PDF Files.zip


In [ ]:
print("Upload the job description (as a .txt file)")
job_desc_file = files.upload()
job_desc_path = list(job_desc_file.keys())[0]

with open(job_desc_path, 'r', encoding='utf-8') as f:
    job_description = f.read()


👉 Upload the job description (as a .txt file)


Saving Job discription.txt to Job discription.txt


In [5]:
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load and extract all resume texts
resume_texts = []
resume_names = []

for filename in os.listdir("resumes"):
    if filename.endswith(".pdf"):
        full_path = os.path.join("resumes", filename)
        text = extract_text_from_pdf(full_path)
        resume_texts.append(text)
        resume_names.append(filename)


In [6]:
def rank_resumes(job_desc, resumes_text):
    documents = [job_desc] + resumes_text
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(documents)
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    return similarities

scores = rank_resumes(job_description, resume_texts)

# Show results
ranked = sorted(zip(resume_names, scores), key=lambda x: x[1], reverse=True)

print("\n📊 Ranked Resumes by Match Score:\n")
for name, score in ranked:
    print(f"{name}: {score:.2f}")



📊 Ranked Resumes by Match Score:

Document from -__mr___rio__-2.pdf: 0.06
Document from -__mr___rio__-4.pdf: 0.03
Document from -__mr___rio__-3.pdf: 0.03
